In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [992 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-03 07:02:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-11-03 07:02:20 (11.1 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("amazon-tools-challenge16").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [ ]:
Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
from pyspark.sql.functions import count
customers_table = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_table.show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|45978717   |2             |
|43622307   |1             |
|740678     |4             |
|29045703   |1             |
|52484548   |3             |
|42560427   |4             |
|17067926   |1             |
|10093406   |1             |
|44979559   |1             |
|19432125   |1             |
|26079415   |1             |
|29931671   |1             |
|12945150   |3             |
|12036434   |1             |
|14230926   |1             |
|45015535   |1             |
|39064792   |1             |
|20709090   |1             |
|45074906   |1             |
|26767269   |2             |
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_table = df.select(["product_id","product_title"]).drop_duplicates()
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_table = review_id_table.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|   15785389|B00H5U9ZD6|     115362950| 2015-08-31|
|  RF0D1LEIF6L7|   47910848|B001TJGCS0|     570955425| 2015-08-31|
|  RM6YKIWQVNSY|   36328996|B000NIK8JW|     128843593| 2015-08-31|
|R1RL3L68ASPS36|   51785809|B008ZYRGUA|     407828107| 2015-08-31|
|R1U4XFBFAG34CY|   40757491|B00K5CA0GC|     490746675| 2015-08-31|
|R3KFIK8P0I91PL|   35544833|B00AIJAA94|     148352067| 2015-08-31|
| RENOAY76PPK1O|   16474909|B00JKEI6K8|     331801084| 2015-08-31|
| RINV884I0NL5V|   22601598|B00AGCHVS6|     471514859| 2015-08-31|
| R5KJH6CXZH2PX|   16129808|B002S0O7U4|     162253576| 2015-08-31|
| RO69JF6QWD0W1|   24382335|B0084YHXMW|      69530650| 2015-08-31|
|R3L9NQBH3FI55C|   49796324|B00MLSS1SW|     916693555| 2015-08-31|
| R4YH95YPHVU0C|   33289687|B00D4WLS2A|      39333316| 2015-08

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [ ]:
Connect to the AWS RDS instance and write each DataFrame to its table.

In [ ]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://amazon-toolsdb.cccvmqkd3tag.us-east-1.rds.amazonaws.com:5432/amazon_tools_db"
config = {"user":"root", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
# 10 minutes 55 seconds
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# 32 seconds
products_table.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
# 3 minutes 26 seconds
customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 7 seconds minutes
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored